# Convolutional Neural Networs

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/riccardoberta/machine-learning/blob/master/04-deep-learning/03-convolutional-neural-networks.ipynb)

Convolutional neural networks (CNNs) emerged from the study of the **brain’s visual cortex**. They power image search services, self-driving cars, automatic video classification systems, voice recognition and natural language processing.

1. [Human Visual Cortex](#Human-Visual-Cortex)
2. [Convolutional Layers](#Convolutional-Layers)



In [2]:
import sklearn
import tensorflow as tf
import numpy as np
import pandas as pd
import os

## Human Visual Cortex

[David H. Hubel and Torsten Wiesel](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1363130/pdf/jphysiol01298-0128.pdf) performed a series of experiments on cats and monkeys in the '50s, giving crucial insights into the structure of the visual cortex. In particular, they showed that many neurons in the visual cortex have a small local receptive field, meaning they react only to visual stimuli located in a limited region of the visual field. The receptive fields of different neurons may overlap, and together they tile the whole visual field. Moreover, they showed that some neurons react only to images of horizontal lines, while others react only to lines with different orientations (two neurons may have the same receptive field but react to different line orientations). They also noticed that some neurons have larger receptive fields, and they react to more complex patterns that are combinations of the lower-level patterns. These observations led to the idea that the higher-level neurons are based on the outputs of neighboring lower-level neurons. 

<img src="./images/visual-cortex.png" width="500">


This powerful architecture is able to detect all sorts of complex patterns in any area of the visual field.

These studies of the visual cortex inspired the **neocognitron**, introduced in
1980, which gradually evolved into what we now call convolutional neural networks. An important milestone was a 1998 paper by [Yann LeCun et al. **Gradient-Based Learning Applied to Document Recognition**](https://ieeexplore.ieee.org/document/726791) that introduced the **LeNet-5** architecture, widely used by banks to recognize handwritten check numbers. This architecture has some building blocks that we already know, such as fully connected layers and sigmoid activation functions, but it also introduces two new building blocks: **convolutional layers** and **pooling layers**.

But, why not simply use a deep neural network with fully connected layers for image recognition tasks? Although this works fine for small images (e.g.,
MNIST), it breaks down for larger images because of the huge number of parameters it requires. For example, a 100x100–pixel image has 10,000 pixels, and if the first layer has just 1,000 neurons, this means a total of 10 million connections. And that’s just the first layer. CNNs solve this problem using partially connected layers and weight sharing.

## Convolutional Layers

The most important building block of a CNN is the convolutional layer: neurons in the first convolutional layer are not connected to every single pixel in the input image, but only to pixels in their receptive fields. In turn, each neuron in the second convolutional layer is connected only to neurons located within a small rectangle in the first layer. This architecture allows the network to concentrate on small low-level features in the first hidden layer, then assemble them into larger higher-level features in the next hidden layer, and so on.

<img src="./images/convolutional-layers.png" width="500">

All the multilayer neural networks we’ve looked at so far had layers composed of a long line of neurons, and we had to flatten input images to 1D before feeding them to the neural network. In a CNN each layer is represented in 2D, which makes it easier to match neurons with their corresponding inputs.

A neuron located in row $i$, column $j$ of a given layer is connected to the outputs of the neurons in the previous layer located in rows $i$ to $i+f_h-1$, columns $j$ to $j+f_w-1$, where $f_h$ and $f_w$ are the height and width of the receptive field. In order for a layer to have the same height and width as the previous layer, it is common to add zeros around the inputs. This is called **zero padding**.

<img src="./images/zero-padding.png" width="500">

It is possible to connect a large input layer to a much smaller layer by spacing out the receptive fields. This reduces the model’s computational complexity. The shift from one receptive field to the next is called the **stride** . 

<img src="./images/stride.png" width="500">

In the figure, a $5 \times 7$ input layer with  zero padding is connected to a $3 \times 4$ layer, using $3 \times 3$ receptive fields and a stride of $2$ in both directions. A neuron located in row $i$, column $j$ in the upper layer is connected to the outputs of the neurons in the previous layer located in rows $i \times s_h$ to $i \times s_h + f_h – 1$, columns $j \times s_w$ to $j  \times s_w + f_w – 1$, where $s_h$ and $s_w$ are the vertical and horizontal strides.

### Kernels

The weights of a neuron (called **filter** or **convolutional kernel**) can be represented as a small image with the size of the receptive field. For example, the following figure shows two possible sets of weights:

<img src="./images/kernels.png" width="500">

The first one is a black square with a vertical white line in the middle, neurons using these weights will ignore everything in their receptive field except for the central vertical line. The second filter is a black square with a horizontal white line in the middle. Once again, neurons using these weights will ignore everything in their receptive field except for the central horizontal line. 

A layer full of neurons using the same filter outputs a **feature map**, which highlights the areas in an image that activate the filter the most. Of course, we do not have to define the filters manually: instead, during training the convolutional layer will automatically learn the most useful filters for its task, and the layers above will learn to combine them into more complex patterns.

Up to now, for simplicity, we have represented the output of each convolutional layer as a 2D layer, but in reality a convolutional layer has multiple filters (we decide how many) and outputs one feature map per filter, so it is more accurately represented in 3D. Moreover, input images are also composed of multiple sublayers: one per color channel. There are typically three: red, green, and blue (RGB). Grayscale images have just one channel, but some images may have much more—for example, satellite images that capture extra light frequencies (such as infrared).

<img src="./images/feature-maps.png" width="500">

It has one neuron per pixel in each feature map, and all neurons within a given
feature map share the same parameters (i.e., the same weights). Neurons in different feature maps use different parameters. A neuron’s receptive field is the same as described earlier, but it extends across all the previous layers’ feature maps. In short, a convolutional layer simultaneously applies multiple trainable filters to its inputs, making it capable of detecting multiple features anywhere in its inputs. The fact that all neurons in a feature map share the same parameters dramatically reduces the number of parameters in the model. Once the CNN has learned to recognize a pattern in one location, it can recognize it in any other location. 

Specifically, a neuron located in row $i$, column $j$ of the feature map $k$ in a given convolutional layer $l$ is connected to the outputs of the neurons in the previous layer $l–1$, located in rows $i \times s_h$ to $i \times s_h + f_h – 1$, columns $j \times s_w$ to $j  \times s_w + f_w – 1$, across all feature maps (in layer $l–1$). Note that all neurons located in the same row and column but in different feature maps are connected to the outputs of the exact same neurons in the previous layer. In a formula, the weighted sum of all the inputs, plus the bias term is:

$z_{i,j,k}=b_k+\sum\limits_{u=0}^{f_{h-1}}{\sum\limits_{v=0}^{f_{w-1}}{\sum\limits_{u=0}^{f_{h-1}}{}}}$
